In [7]:
import sys
from xmlrpc.server import SimpleXMLRPCServer, SimpleXMLRPCRequestHandler
import socketserver
import xmlrpc.client
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import matplotlib.pyplot as plt


from sklearn.model_selection import ParameterGrid

In [16]:
HY_COMBOS = {
     # each element of conv_layers should be tuple:
     #(num filters, filtersize tuple, poolsize tuple OR none if not pooling)
    "conv_layers": [[(32,(3,3),(2, 2)), (10,(3,3),(2,2))], [(32,(3,3),(2, 2))],[]],
    # this is a list of sizes for each dense layer
    # do not include the final prediction layer here
    "dense_layers" : [[50,10,5], [40,20],[10],[]],
    # apply the SAME activation and dropout to every conv and dense layer 
    # (except for final output which has hardcoded softmax activation w/ no dropout)
    "activation": ["relu", "sigmoid"], 
    "dropout":  np.arange(0,.5,.1),
    # l2 regularization for weights
    "k_reg": [0,.001],
    # optimization hyperparameters
    "learning_rate": [.01],
    "grad_clip_norm": np.arange(0,.5,.2)}
    
    
hy_list = list(ParameterGrid(HY_COMBOS)) 
print("Total combos:", len(hy_list))

Total combos: 720


In [17]:
max_epochs = 1000

In [21]:
losses = np.empty([len(hy_list), max_epochs])
losses.fill(np.nan)

In [24]:
def update(epoch_num, model_num, loss):
    losses[model_num, epoch_num] = loss
    return



In [ ]:
# if we pick best model based on best performance across training
best_model = np.where(losses == np.nanmin(losses))[0][0]





# if we pick best model based on the last training epoch
def get_last_loss(row):
    if len(np.where(~np.isnan(row))[0]):
        return (row[np.where(~np.isnan(row))[0][-1]])
    else:
        return np.nan

lastlosses = np.array([get_last_loss(row) for row in losses])
best_model = np.where(lastlosses == np.nanmin(lastlosses))[0][0]
